# PROJECT - *My Way* of seeing music covers
#### Pierre-Antoine Desplaces, Anaïs Ladoy, Lou Richard

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from io import StringIO
import sys
import requests
from bs4 import BeautifulSoup
import pickle
import os
import glob
from pathlib import Path

## Notebook plan
1. Data importation
2. Clique organisation (Multi-level indexing)
3. Addition of the language and the year of each track (SHS website web-scraping)
4. Addition of the tempo and song hotness of each track (Access to track files through the cluster)
5. Determine artist location for spatial analysis
6. Addition of the genre for each track (Use of LastFM dataset and external website for genre listing)
7. Distinction of the original song and the music covers inside each clique

### 1. Data importation
Download available additional files containing metadata about our dataset from the cluster (dataset/million-songs_untar/)
- tracks_per_year.txt
- unique_tracks.txt
- unique_artists.txt
- artist_location.txt

Use the Second Hand Songs (SHS) dataset that was created through a collaboration between the Million Songs team and the Second Hand Songs website (https://secondhandsongs.com/). These data are splitted into two datasets to allowed machine learnings algorithms (a train and a test set).
- SHS_testset.txt
- SHS_trainset.txt
Since we don't need this distinction for our data analysis, we merged these two datasets.

The use of external dataset (LastFM) for the genres and the use of the track files (.h5) available through the cluster are commented in part 4 and 5.

Some general informations about our data :
- All the additional files were downloaded from the cluster giving all the metadata of the Million Songs dataset. They will help to elaborate a plan and a script will then search more information about a specific track (h5 files in the cluster) maybe using cluster cpu. The path to access to a track in the cluster is for example million-songs/data/A/A/A (with the 3 letters at the end being the 3rd, 4th and 5th letter on the track id).
- The music covers will be detected using another dataset (SecondHandSongs), we have the choice to use the downloadable dataset containing 18,196 tracks (all with a connection to the MSD dataset), or to web-scrapp the SHS website (https://secondhandsongs.com/) where we have much more information (522 436 covers) but not necessarly connected to our MSD dataset. The SHS API is RESTful (return a JSON object) and will be used to provide additional or missing informations (localisation, language of the song, ...) in our dataset.
- Some artist are geolocalised (30% of the MSD total artists) on the artist_location dataframe.

In [2]:
#Load Additional files
tracks_per_year=pd.read_csv('data/AdditionalFiles/tracks_per_year.txt',delimiter='<SEP>',engine='python',header=None,index_col=1,names=['year','trackID','artist','title'])
unique_tracks=pd.read_csv('data/AdditionalFiles/unique_tracks.txt',delimiter='<SEP>',engine='python',header=None,index_col=0,names=['trackID','songID','artist','title'])
unique_artists=pd.read_csv('data/AdditionalFiles/unique_artists.txt',delimiter='<SEP>',engine='python',header=None,index_col=0,names=['artistID','artistMID','randomTrack','name'])
artist_location=pd.read_csv('data/AdditionalFiles/artist_location.txt',delimiter='<SEP>',engine='python',header=None,index_col=0,names=['artistID','lat','long','name','location'])

In [3]:
#Check if indexes is unique and print the number of elements for each dataframe
print('Dataframe (Unique index, Number of elements)')
print('tracks_per_year ',(tracks_per_year.index.is_unique,tracks_per_year.shape[0]))
print('unique_tracks ',(unique_tracks.index.is_unique,unique_tracks.shape[0]))
print('unique_artists ',(unique_artists.index.is_unique,unique_artists.shape[0]))
print('artist_location ',(artist_location.index.is_unique,artist_location.shape[0]))

Dataframe (Unique index, Number of elements)
tracks_per_year  (True, 515576)
unique_tracks  (True, 1000000)
unique_artists  (True, 44745)
artist_location  (True, 13850)


The covers dataset (SHS_testset.txt and SHS_trainset.txt) were organised in a very special way where group (named "cliques") list some tracks that are interrelated (music covers and original track). The function **read_shs_files** is used to import the files keeping the "clique" configuration.

In [4]:
def read_shs_files(pathToFile):
    f = open(pathToFile)
    s = StringIO()
    cur_ID = None
    for ln in f:
        if not ln.strip():
                continue
        if ln.startswith('%'):
                cur_ID = ln.replace('\n','<SEP>',1)
                continue
        if cur_ID is None:
                print ('NO ID found')
                sys.exit(1)
        s.write(cur_ID + ln)
    s.seek(0)
    df = pd.read_csv(s,delimiter='<SEP>',engine='python',header=None,names=['shsID','trackID','artistID','shsPerf'])
    return df

In [5]:
#Import the two SHS datasets and concatenate them
SHS_testset=read_shs_files('data/SHS_testset.txt')
SHS_trainset=read_shs_files('data/SHS_trainset.txt')
covers=pd.concat([SHS_testset,SHS_trainset])
covers.shsID=covers.shsID.str.strip('%')
covers.head()

,shsID,trackID,artistID,shsPerf
0,"115402,74782, Putty (In Your Hands)",TRJVDMI128F4281B99,AR46LG01187B98DB5D,74784
1,"115402,74782, Putty (In Your Hands)",TRNJXCO128F92E1930,ARQD13K1187B98E441,138584
2,"24350, I.G.Y. (Album Version)",TRIBOIS128F9340B19,ARUVZYG1187B9B2809,24350
3,"24350, I.G.Y. (Album Version)",TRGXZDU128F9301E53,AR4LE591187FB3FCFB,24363
4,"79178, When The Catfish Is In Bloom",TRQSIOY128F92FACA7,ARU75JD1187FB38B79,79178


As said before, our dataset was created from a collaboration between the Million Songs Dataset (MSD) and the Second Hand Songs Dataset (SHS).  

The MSD consists of almost all the information available through the Echo Nest API for one million popular tracks and the *trackID* and *artistID* are directly based on the Echo Nest structure.   
More specifically, the *trackID* is the unique identifier of a track (connection with unique_tracks.txt) and the *artistID* is the unique identifier of an artist (connection with unique_artists.txt). The *trackID* is also the path used to navigate through the MSD directory and access to a specific song (through the cluster).  

The *shsPerf* corresponds to the SHS identifier of a song and this information is sometimes missing. Exploring the SHS website, we found that the page of a specific song can be accessed with different paths using the *shsPerf* information :
- https://secondhandsongs.com/performance/ [shsPerf]
- https://secondhandsongs.com/work/ [shsPerf]

We have deducted that only original music song has both a work page and a performance page, covers having only performance page. In some cases, the *shsPerf* for an original music song is not the same if we want to reach the performance page or the work page and in our dataset, we assumed that the *shsPerf* was only performance id.

The structure of the first column was impossible to decode and the informations contained as the name of the song can be found elsewhere (unique_tracks.txt) thus, this column will be removed.

### 2. Clique organisation (Multi-level indexing)
As said before, the cover dataset is organised as cliques that group some specific tracks that are interrelated. 
In order to keep this structure, we decide to use a multilevel index with cliques (need to transform shsID category in int) and then use the ranking according the released date of the track (year attribute) for the second index (thus, 0 will be the original song).

In [6]:
#Convert shsID to clique id (first convert to category and get a code)
covers=covers.assign(clique_id=(covers.shsID.astype('category')).cat.codes)
#Remove the shsID column (useless since we have the clique_id now)
covers.drop('shsID',axis=1,inplace=True)

In order to use the informations contained in the metadata files first, we merged some necessary attributes (name of the artist, title of the track, released date) from the MSD dataframes named before.

In [7]:
#Merge with unique_artists dataframe to find the artist name for each track (no taking consideration of featuring since we take only the name of the artist assigned with the track)
covers=covers.merge(unique_artists[['name']],how='left',left_on='artistID',right_index=True)
#Merge with unique_tracks dataframe to find the track name
covers=covers.merge(unique_tracks[['title']],how='left',left_on='trackID',right_index=True)
#Merge with tracks_per_year dataframe to find the year of each track
covers=covers.merge(tracks_per_year[['year']],how='left',left_on='trackID',right_index=True)

In [8]:
covers.head()

,trackID,artistID,shsPerf,clique_id,name,title,year
0,TRJVDMI128F4281B99,AR46LG01187B98DB5D,74784,1433,The Detroit Cobras,Putty (In Your Hands),1998.0
1,TRNJXCO128F92E1930,ARQD13K1187B98E441,138584,1433,Sylvie Vartan,Ne Le Déçois Pas,1962.0
2,TRIBOIS128F9340B19,ARUVZYG1187B9B2809,24350,2543,Donald Fagen,I.G.Y. (Album Version),1982.0
3,TRGXZDU128F9301E53,AR4LE591187FB3FCFB,24363,2543,Take 6,Beautiful World (Album Version),NaN
4,TRQSIOY128F92FACA7,ARU75JD1187FB38B79,79178,5240,John Fahey,When The Catfish Is In Bloom,1968.0


Here is printed some useful informations about the cover dataset (the basis of our work) :

In [9]:
print('Number of tracks :', covers.shape[0])
print('Number of cliques :', max(covers.index)+1) #Number of cliques (+1 because id starts at 0)
print('Number of unique tracks :', len(covers.trackID.unique())) 
print('Number of unique artists :', len(covers.artistID.unique()))
print('Number of missing trackID :', len(covers[covers.trackID.isnull()]))
print('Number of missing artistID :', len(covers[covers.artistID.isnull()]))
print('Number of missing years :', len(covers[covers.year.isnull()]))

Number of tracks : 18196
Number of cliques : 12960
Number of unique tracks : 18196
Number of unique artists : 5578
Number of missing trackID : 0
Number of missing artistID : 0
Number of missing years : 4796


In [10]:
covers=covers.sort_values(['clique_id', 'year'], ascending=[True, True]).reset_index() #Reset index according clique_id and year
covers.drop('index',axis=1,inplace=True) #Drop the previous index

### 3. Addition of the language and the year of each track (SHS website web-scraping)
With the results printed above, we noticed that 4796 years were missing (26%) and since we thought to detect the original song by ranking the tracks year in each clique, we needed to find a way to get them.
Furthermore, year isn't necessarly sufficient informations to discriminate the tracks (cover appears sometimes in the same year than the original one), thus it will be better to have the released date for ALL the tracks if the information is available in the SHS website.

Indeed, the Second Hand Songs website allows API request on their database (limited to 1000 requests per hour).
Each track has a performance page where we can have access additional informations about the track as the language, the released date and the original song of the specific cover. In the SHS website, the performance id (that is used in the URL to access to the performance page) is available in our cover dataframe (shsPerf).
Nevertheless, we have negative values of shsPerf and it corresponds to missing values.

Thus, we have two ways to access extract the language/year/original song via web-scrapping :
- For valid SHS performance ID, access to the performance page (e.g. 'https://secondhandsongs.com/performance/1983') and web-scrapping of the Language and Released date informations using the perfInfo() function.
- For invalid SHS performance ID, API request to the search page (e.g. 'https://secondhandsongs.com/search/performance?title=blackbird&performer=beatles'), extract the perf ID with the find_PerfID() and then use the perfInfo() function.

In [11]:
print('Number of missing years with valid shsPerf (API request on the performance page) :',len(covers[(covers.year.isnull()) & (covers.shsPerf != -1)]))
print('Number of missing years with invalid shsPerf (API request on the search page to find shsPerf) :',len(covers[(covers.year.isnull())])-len(covers[(covers.year.isnull()) & (covers.shsPerf != -1)]))

Number of missing years with valid shsPerf (API request on the performance page) : 4128
Number of missing years with invalid shsPerf (API request on the search page to find shsPerf) : 668


In [12]:
pickle.dump(covers,open('covers.p','wb'))

In order to test these algorithms, we have worked for now with a part of the cover dataframe (part dataframe), containing only 962 tracks but being representative of the covers dataframe.

In [13]:
covers.shape

(18196, 7)

In [14]:
#Merge with the unique_tracks dataframe to get the name of the artist for the track (take featuring as well), it will be useful for the find_shsPerf function 
covers=covers.merge(unique_tracks[['artist']],how='left',left_on='trackID',right_index=True)
covers.head()

,trackID,artistID,shsPerf,clique_id,name,title,year,artist
0,TRGDMZP128F42BC52B,ARB1DDF1187FB4FCFB,-1,0,Louis Armstrong,Stardust,1988.0,Louis Armstrong & His Orchestra
1,TRCATYW12903D038FE,ARGJEEO1271F573FD6,-1,0,Artie Shaw and his orchestra,Stardust,1988.0,Artie Shaw and his orchestra
2,TRVMZJZ128F4270CE4,ARY0HTV1187FB4A1B1,-1,0,Hoagy Carmichael,Star Dust,1999.0,Hoagy Carmichael
3,TRKOINL128F42926C3,ARQ5FSZ1187B98AD74,-1,0,Connee Boswell & Sy Oliver Orchestra,Star Dust,NaN,Connee Boswell & Sy Oliver Orchestra
4,TROJZTF128F428B546,ARJN76O1187FB43C99,-1,1,Ana Belén,Yo Vengo A Ofrecer Mi Corazon,2001.0,Ana Belén


In [15]:
print('Number of cliques in the subset :', len(covers.clique_id.unique()))
print('Number of tracks in the subset :', covers.shape[0])
print('Number of missing years in the subset :', len(covers[covers.year.isnull()]))
print('Number of invalid shsPerf in the subset :', len(covers[covers.shsPerf<0]))

Number of cliques in the subset : 5854
Number of tracks in the subset : 18196
Number of missing years in the subset : 4796
Number of invalid shsPerf in the subset : 3075


In [16]:
covers_p=covers[1:20]

In [17]:
covers_p

,trackID,artistID,shsPerf,clique_id,name,title,year,artist
1,TRCATYW12903D038FE,ARGJEEO1271F573FD6,-1,0,Artie Shaw and his orchestra,Stardust,1988.0,Artie Shaw and his orchestra
2,TRVMZJZ128F4270CE4,ARY0HTV1187FB4A1B1,-1,0,Hoagy Carmichael,Star Dust,1999.0,Hoagy Carmichael
3,TRKOINL128F42926C3,ARQ5FSZ1187B98AD74,-1,0,Connee Boswell & Sy Oliver Orchestra,Star Dust,NaN,Connee Boswell & Sy Oliver Orchestra
4,TROJZTF128F428B546,ARJN76O1187FB43C99,-1,1,Ana Belén,Yo Vengo A Ofrecer Mi Corazon,2001.0,Ana Belén
5,TRYQEDQ128F427917C,ARS4KT21187B9B9438,-1,1,Fito Paez,Yo Vengo A Ofrecer Mi Corazon,NaN,Fito Paez
6,TRCKNGE128F92DA3F3,AR1CB5G1187B9AFB8E,-1,2,Electric Light Orchestra,Mr. Blue Sky,1977.0,Electric Light Orchestra
7,TRIOPLY128F423CFF3,ARKZJ301187FB521B2,-1,2,Lily Allen,Mr Blue Sky,2007.0,Lily Allen
8,TRWNDEU128F9329BF7,ARVZWQ31187B9B8946,-1,3,Liars,Mr Your On Fire Mr,2002.0,Liars
9,TRYOPHS128F146DEFD,AR6NYHH1187B9BA128,-1,3,Yeah Yeah Yeahs,Mr. You're On Fire Mr.,2003.0,Yeah Yeah Yeahs
10,TRMBSQR128F92DF66E,ARPI2DX1187FB4CED4,-1,4,Andrea True Connection,More More More,1991.0,Andrea True Connection


In [18]:
#API request to find the SHS perf for the unvalid ones (negative values)
def find_shsPerf(x):
    
    title=covers.iloc[x]['title']
    artist=covers.iloc[x]['name']
    shsPerf=covers.iloc[x]['shsPerf']
    
    if shsPerf<0:
        title=title.replace('.', '').replace('_', '').replace('/', '').lower().replace(' ','+')
        artist=artist.replace('.', '').replace('_', '').replace('/', '').lower().replace(' ','+')
        r=requests.get('https://secondhandsongs.com/search/performance?title='+title+'&op_title=contains&performer='+artist+'&op_performer=contains')
        soup = BeautifulSoup(r.text, 'html.parser')
        results=soup.find('tbody')

        if results is None :
            new_shsPerf=0
        else:
            new_shsPerf=int(results.find('a',attrs={'class':'link-performance'})['href'].split('/')[2])
    else :
        new_shsPerf=shsPerf
        
    if x%500==0:
        print(x)
        
    return new_shsPerf

In [19]:
#Find the shsPerf for the tracks which doesn't have valid ones (substract 2055 to part dataframe index to start with index=0)
#covers_withSHS=covers_p.shsPerf.index.map(lambda x: find_shsPerf(x)) 

In [20]:
#covers_withSHS=pd.concat([part1, part2, part3, part4, part5, part6])

In [21]:
#pickle.dump(covers_withSHS,open('data/covers_withSHS_new.p','wb'))
covers_withSHS=pickle.load(open("data/covers_withSHS_new.p","rb"))

In [22]:
covers['shsPerf']=covers_withSHS

In [23]:
len(covers[covers.shsPerf==0])

1088

We still can't access the SHS pages for **1050** music covers. We will need to decide if remove them since it will be impossible to find the missing release date and/or the language of the track.
For now, we will compute the perfInfo_SHS for all the dataset.

In [24]:
#API request to SHS website for the page of a specific performance (defined as shsPerf) to extract Language and Date
def perfInfo_SHS(shsPerf):
        
    if shsPerf==0:
        perfLanguage='Unavailable'
        perfDate='Unavailable'
        original_shsPerf='Unavailable'
        
    else :
        r = requests.get('https://secondhandsongs.com/performance/'+str(shsPerf))
        soup = BeautifulSoup(r.text, 'html.parser')
        perfMeta=soup.find('dl',attrs={'class':'dl-horizontal'})
        if perfMeta is None:
            perfLanguage='Missing'
            perfDate='Missing'
            original_shsPerf='Missing'
        else :
            perfLanguage=perfMeta.find('dd',attrs={'itemprop':'inLanguage'})
            if perfLanguage is None :
                perfLanguage='Missing'
            else :
                perfLanguage=perfLanguage.text

            #Find the performance date    
            perfDate=perfMeta.find('div',attrs={'class':'media-body'})
            if perfDate is None :
                perfDate='Missing'
            else :
                perfDate=perfDate.find('p').text.split('\n')[2].strip(' ')

            #Find the original shsPerf (work or performance ID) 
            original_section=soup.find('section',attrs={'class':'work-originals'})
            versions_section=soup.find('section',attrs={'id':'entity-section'})
            
            if original_section is None :
                if versions_section is None :
                    print('2')
                    original_shsPerf='Missing'
                else :
                    
                    original_shsPerf=versions_section.find('a')['href']
                    if original_shsPerf is None :
                        print('3')
                        original_shsPerf='Missing'
                    else :
                        print('4')
                        original_shsPerf=original_shsPerf.split('/')[2]

            else :
                original_shsPerf=original_section.find('div',attrs={'class':'media-body'})
                original_shsWork=original_section.find('a',attrs={'class':'link-work'})['href']
                
                if original_shsPerf is None :
                    print('5')
                    original_shsPerf=original_shsWork.split('/')[2]
                else :
                    print('6')
                    original_shsPerf=original_shsPerf.find('a')['href'].split('/')[2]

    return perfLanguage,perfDate,original_shsPerf

In [25]:
#covers['language'], \
#covers['date'], \
#covers['original_shsPerf']= zip(*covers.shsPerf.map(perfInfo_SHS))

In [26]:
covers.tail()

,trackID,artistID,shsPerf,clique_id,name,title,year,artist
18191,TRUZAVE128F426E391,AR3DLBB1187B98F3DF,106651,5852,The Statler Brothers,Oh Happy Day,1993.0,The Statler Brothers
18192,TRDNIRY128F425A5C8,ARDRJSP126E2B3BEF8,248918,5852,Spiritualized,Oh Happy Day,1998.0,Spiritualized
18193,TRPVDKP128F934956E,ARELPXQ1187FB384FD,138704,5852,Queen Latifah,Oh Happy Day,NaN,Queen Latifah with Jubilation
18194,TREMVLC128F92EFA95,ARBFDJW1187B9AD27A,29576,5853,The Fabulous Thunderbirds,Tip On In,1987.0,The Fabulous Thunderbirds
18195,TRMTMWV128F92F6F78,ARDYR3C1187FB461CE,9986,5853,George Thorogood And The Destroyers,Tip On In,NaN,George Thorogood & The Destroyers


In [27]:
#pickle.dump(covers,open('data/covers_new.p','wb'))
covers=pickle.load(open("data/covers_new.p","rb"))

In [28]:
covers.shape

(18196, 11)

In [29]:
print('Number of unavailable language : ', len(covers[covers.language=='Unavailable']) )
print('Number of missing language : ', len(covers[covers.language=='Missing']) )
print('Total number of tracks with no language information : ', len((covers[covers.language=='Missing']) | (covers[covers.language=='Unavailable']))) 
print('Number of unavailable original shsPerf :' ,len(covers[covers.original_shsPerf=='Unavailable']))
print('Number of missing original shsPerf :' ,len(covers[covers.original_shsPerf=='Missing']))
print('Total number of tracks with no original shsPerf information : ', len((covers[covers.original_shsPerf=='Missing']) | (covers[covers.original_shsPerf=='Unavailable']))) 

Number of unavailable language :  1088
Number of missing language :  1426
Total number of tracks with no language information :  2514
Number of unavailable original shsPerf : 1088
Number of missing original shsPerf : 145
Total number of tracks with no original shsPerf information :  1233


Title impossible to found 

We noticed several problems with this intermediate result :

1/ The track year (year) is sometimes different form the released date (date) we've extracted from the SHS website, we will prefer the data information found in the SHS website.

In [30]:
covers['date'] = covers.apply(lambda row: row['year'] if ((row['date']=='Missing') | (row['date']=='Unavailable')) else row['date'],axis=1)

In [31]:
covers.drop('year',axis=1,inplace=True)

In [32]:
# Number of tracks where we don't have any information about the release year
len(covers[covers.date.isnull()])

358

In [33]:
covers.shape

(18196, 10)

In [34]:
# Are the web-scrapped original shsPerf are all in the dataset ?
covers[(covers.original_shsPerf != 'Missing') & (covers.original_shsPerf != 'Unavailable')].original_shsPerf.astype('int').isin(covers.shsPerf.unique()).value_counts()

True     9515
False    7448
Name: original_shsPerf, dtype: int64

In [35]:
covers.shsPerf.unique()

array([     0, 412972,  16660, ..., 138704,  29576,   9986])

In [36]:
covers[(covers.original_shsPerf != 'Missing') & (covers.original_shsPerf != 'Unavailable')].original_shsPerf.head()

2     19677
6     16660
7     16660
8    142889
9    354066
Name: original_shsPerf, dtype: object

In [37]:
# Replace missing values by shsPerf and unavailable by 0
covers['original_shsPerf'] = np.where(((covers['original_shsPerf']=='Unavailable') | (covers['original_shsPerf']=='Missing')), 0, covers['original_shsPerf'])
covers['original_shsPerf']=covers['original_shsPerf'].astype(int)

In [38]:
covers.head()

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf
0,TRGDMZP128F42BC52B,ARB1DDF1187FB4FCFB,0,0,Louis Armstrong,Stardust,Louis Armstrong & His Orchestra,Unavailable,1988,0
1,TRCATYW12903D038FE,ARGJEEO1271F573FD6,0,0,Artie Shaw and his orchestra,Stardust,Artie Shaw and his orchestra,Unavailable,1988,0
2,TRVMZJZ128F4270CE4,ARY0HTV1187FB4A1B1,412972,0,Hoagy Carmichael,Star Dust,Hoagy Carmichael,English,1942,19677
3,TRKOINL128F42926C3,ARQ5FSZ1187B98AD74,0,0,Connee Boswell & Sy Oliver Orchestra,Star Dust,Connee Boswell & Sy Oliver Orchestra,Unavailable,NaN,0
4,TROJZTF128F428B546,ARJN76O1187FB43C99,0,1,Ana Belén,Yo Vengo A Ofrecer Mi Corazon,Ana Belén,Unavailable,2001,0


In [39]:
# Compute the frequency for each original shsPerf and sort values according frequency
freq_original=covers.groupby(['clique_id','original_shsPerf'],as_index=False)['clique_id'].agg({'freq':'count'})

In [40]:
freq_original.sort_values(['clique_id', 'freq'], ascending=[True, False],inplace=True)
freq_original.set_index(['clique_id', 'freq'],drop=False,inplace=True)
freq_original.drop('clique_id',axis=1,inplace=True)

freq_original.head()

original_shsPerf  freq
clique_id freq                        
0         3                    0     3
          1                19677     1
1         2                    0     2
2         2                16660     2
3         1               142889     1

In [41]:
len(covers.shsPerf.unique()[1:])

16974

In [42]:
freq_original.dtypes

original_shsPerf    int64
freq                int64
dtype: object

In [43]:
freq_original_agg=freq_original[['freq']].groupby(by='clique_id').agg([{'nbrow':'count','nbfreq':'sum'}])

/Users/anaisladoy/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:2946: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  results[name] = obj.aggregate(func)


In [44]:
freq_original_agg.columns=freq_original_agg.columns.droplevel()
freq_original_agg.columns=freq_original_agg.columns.droplevel()
freq_original_agg['clique_id']=freq_original_agg.index

In [45]:
freq_original_agg.head()

,nbrow,nbfreq,clique_id
clique_id,,,
0,2,4,0
1,1,2,1
2,1,2,2
3,2,2,3
4,1,2,4


In [46]:
#Keep only clique where there are 2 elements with two different original_shsPerf
freq_original_agg=freq_original_agg[(freq_original_agg.nbrow==2) & (freq_original_agg.nbfreq==2)]

In [47]:
freq_original_agg.head()

,nbrow,nbfreq,clique_id
clique_id,,,
3,2,2,3
6,2,2,6
8,2,2,8
12,2,2,12
13,2,2,13


In [48]:
covers_clique=covers.copy()
covers_clique.set_index(['clique_id','trackID'],inplace=True)

In [49]:
covers_clique.head()

artistID  shsPerf  \
clique_id trackID                                           
0         TRGDMZP128F42BC52B  ARB1DDF1187FB4FCFB        0   
          TRCATYW12903D038FE  ARGJEEO1271F573FD6        0   
          TRVMZJZ128F4270CE4  ARY0HTV1187FB4A1B1   412972   
          TRKOINL128F42926C3  ARQ5FSZ1187B98AD74        0   
1         TROJZTF128F428B546  ARJN76O1187FB43C99        0   

                                                              name  \
clique_id trackID                                                    
0         TRGDMZP128F42BC52B                       Louis Armstrong   
          TRCATYW12903D038FE          Artie Shaw and his orchestra   
          TRVMZJZ128F4270CE4                      Hoagy Carmichael   
          TRKOINL128F42926C3  Connee Boswell & Sy Oliver Orchestra   
1         TROJZTF128F428B546                             Ana Belén   

                                                      title  \
clique_id trackID                                             
0         TRGDMZP128F42BC52B                       Stardust   
          TRCATYW12903D038FE                       Stardust   
          TRVMZJZ128F4270CE4                      Star Dust   
          TRKOINL128F42926C3                      Star Dust   
1         TROJZTF128F428B546  Yo Vengo A Ofrecer Mi Corazon   

                                                            artist  \
clique_id trackID                                                    
0         TRGDMZP128F42BC52B       Louis Armstrong & His Orchestra   
          TRCATYW12903D038FE          Artie Shaw and his orchestra   
          TRVMZJZ128F4270CE4                      Hoagy Carmichael   
          TRKOINL128F42926C3  Connee Boswell & Sy Oliver Orchestra   
1         TROJZTF128F428B546                             Ana Belén   

                                 language  date  original_shsPerf  
clique_id trackID                                                  
0         TRGDMZP128F42BC52B  Unavailable  1988                 0  
          TRCATYW12903D038FE  Unavailable  1988                 0  
          TRVMZJZ128F4270CE4      English  1942             19677  
          TRKOINL128F42926C3  Unavailable   NaN                 0  
1         TROJZTF128F428B546  Unavailable  2001                 0

In [50]:
def attribute_original(clique_id) :

    first_elem=covers_clique.loc[clique_id].iloc[0]
    second_elem=covers_clique.loc[clique_id].iloc[1]
    
    if (first_elem.original_shsPerf==0) | (second_elem.original_shsPerf==0) :
        if (first_elem.original_shsPerf==0) & (second_elem.original_shsPerf!=second_elem.shsPerf):
            elemTrack=first_elem.name
            elemSHS=second_elem.original_shsPerf
            
        elif (second_elem.original_shsPerf==0) & (first_elem.original_shsPerf!=first_elem.shsPerf):
            elemTrack=second_elem.name
            elemSHS=first_elem.original_shsPerf
        else :
            elemTrack=np.nan
            elemSHS=np.nan
            
    else :
        elemTrack=np.nan
        elemSHS=np.nan
        
    return elemTrack, elemSHS

In [51]:
replace_shs=pd.DataFrame(columns=['elemTrack','elemSHS'])
replace_shs['elemTrack'], \
replace_shs['elemSHS'] = zip(*freq_original_agg.clique_id.map(attribute_original))

In [52]:
replace_shs.dropna(axis=0,inplace=True)
replace_shs.elemSHS=replace_shs.elemSHS.astype(int)
replace_shs.head()

,elemTrack,elemSHS
1,TRGYREY128E0791913,9133
2,TRYEKWE128F1459C42,68273
9,TRQSJPX128F930DEF2,373587
10,TRSKALN128F931CDD8,82615
14,TRZPKMK128F1462D24,613511


In [53]:
covers=covers.merge(replace_shs,how='left',left_on='trackID',right_on='elemTrack')
covers['shsPerf'] = np.where(covers['elemSHS'].notnull(), covers['elemSHS'], covers['shsPerf'])

In [54]:
covers.drop(['elemTrack','elemSHS'],axis=1,inplace=True)
covers.shsPerf=covers.shsPerf.astype(int)

In [55]:
#Save pickle file
#pickle.dump(covers,open('data/covers_final.p','wb'))
covers=pickle.load(open("data/covers_final.p","rb"))

In [56]:
len(covers[covers.shsPerf==0])

950

In [57]:
def find_original_track(clique_id,max_freq_row):
    
    nb_elems=freq_original.loc[clique_id].original_shsPerf.count()
    unique_shsPerf=covers.shsPerf.unique()[1:];

    if nb_elems == 1 :

        first_elem=max_freq_row.original_shsPerf;
        if first_elem !=0:
            if (first_elem in(unique_shsPerf))==True :
                original_song=first_elem;
            else :
                original_song='Unknown';
        
        else :
            original_song='Unknown';
    
    if nb_elems == 2 :
        first_elem=max_freq_row.original_shsPerf;
        second_elem=freq_original.loc[clique_id].iloc[1].original_shsPerf;
        
        if first_elem != 0:
            if (first_elem in (unique_shsPerf))==True :
                original_song=first_elem;
            elif (second_elem in (unique_shsPerf))==True :
                original_song=second_elem;
            else : #Put the reference as the first
                original_song='Unknown';
        
        else :
            if (second_elem in (unique_shsPerf))==True :
                original_song=second_elem;
            else :
                original_song='Unknown';
    
    
    elif nb_elems>2 :
        first_elem=max_freq_row.original_shsPerf;
        
        second_elem=freq_original.loc[clique_id].iloc[1].original_shsPerf;
        third_elem=freq_original.loc[clique_id].iloc[2].original_shsPerf;
        
        if first_elem != 0:
            if (first_elem in (unique_shsPerf))==True :
                original_song=first_elem;
            elif (second_elem in (unique_shsPerf))==True :
                original_song=second_elem;
            elif (third_elem in (unique_shsPerf))==True :
                original_song=third_elem;
            else :
                original_song='Unknown';
        
        else :
            if (second_elem in (unique_shsPerf))==True :
                original_song=second_elem;
            elif (third_elem in (unique_shsPerf))==True :
                original_song=third_elem;
            else :
                original_song='Unknown';
                
    
    return clique_id, original_song

In [58]:
#Create empty DataFrame to contain the outputs of find_original_track()
original_song_df=pd.DataFrame(columns=['clique_id','original_id'])

In [59]:
original_song_df['clique_id'], \
original_song_df['original_id'] = zip(*pd.Series(freq_original.index.get_level_values('clique_id').unique()).map(lambda x : find_original_track(x,freq_original.loc[x].iloc[0])))

In [60]:
original_song_df.shape

(5854, 2)

In [61]:
#Number of clique where no original songs were found via web-scrapping
len(original_song_df[original_song_df.original_id=='Unknown'])

1944

In [62]:
original_song_df[original_song_df.original_id=='Unknown'].head()

,clique_id,original_id
0,0,Unknown
1,1,Unknown
5,5,Unknown
16,16,Unknown
21,21,Unknown


In [63]:
#Number of original_id duplicates between two cliques
len(original_song_df[(original_song_df.duplicated(subset='original_id',keep=False)) & (original_song_df.original_id!='Unknown')])

98

In [64]:
original_song_df[(original_song_df.duplicated(subset='original_id',keep=False)) & (original_song_df.original_id!='Unknown')].sort_values('original_id').head()

,clique_id,original_id
3584,3584,405
167,167,405
7,7,575
4417,4417,575
1754,1754,623


In [65]:
#Merge the original of each clique with cover dataframe
covers=covers.merge(original_song_df[['clique_id','original_id']],how='left',left_on='clique_id',right_on='clique_id')

In [66]:
dup_shsPerf=covers[covers.duplicated('shsPerf') & covers.shsPerf!=0].sort_values('shsPerf').shsPerf
all_duplicates=covers[covers.duplicated('shsPerf',keep=False) & covers.shsPerf!=0].sort_values('shsPerf')

In [67]:
def duplicate_to_keep(dup_shsPerf) :
    dup=covers[covers.shsPerf==dup_shsPerf]
    dup.replace(['Unknown','Unavailable'],0)
    count=(dup[['language','date','original_shsPerf','original_id']].iloc[:,1:] == 0).sum(axis=1).sort_values()
    trackID_keep=covers.iloc[count.index[0]].trackID
    
    return trackID_keep

In [68]:
to_keep=pd.DataFrame(data=dup_shsPerf.map(duplicate_to_keep))

In [69]:
to_remove=all_duplicates.merge(to_keep,how='left',left_on='trackID',right_on='shsPerf')
to_remove=to_remove[to_remove.shsPerf_y.isnull()]

In [70]:
to_remove.shape

(0, 13)

In [71]:
covers=covers[~covers.trackID.isin(to_remove.trackID)]

In [72]:
covers.shape

(18116, 12)

In [73]:
pickle.dump(covers,open('data/covers_withoutduplicates.p','wb'))
#covers=pickle.load(open("data/covers_withoutduplicates.p","rb"))

In [74]:
def merge_cliques(original_id, covers, cliques):
    cliques_list = cliques[cliques['original_id'] == original_id].index
    c1 = cliques_list[0]
    for c in cliques_list[1:]:
        tracks = covers[covers['clique_id'] == c].index
        for t in tracks:
            covers.set_value(t, 'clique_id', c1)

def merge(covers):
    covers = covers
    cliques = covers.groupby('clique_id').max()
    cliques = cliques[cliques['original_id'] != "Unknown"]
    dup = (cliques[cliques['original_id'].duplicated()].sort_values("original_id")['original_id']).tolist()
    for d in dup:
        merge_cliques(d, covers, cliques)
        
    covers.sort_values("clique_id", inplace=True)
    covers.reset_index(inplace=True)
    covers.drop('index', axis=1, inplace=True)
    return covers

In [78]:
#merge(covers)

In [ ]:
#pickle.dump(covers,open('data/covers_FINAL.p','wb'))
covers=pickle.load(open("data/covers_FINAL.p","rb"))

In [ ]:
#Load the artist location
artist_location=pickle.load(open("data/artists_location.p","rb"))

In [ ]:
artist_location.head()

In [ ]:
covers=covers.merge(artist_location,how='left',left_on='name',right_on='name')

In [ ]:
genres=pickle.load(open("data/genres_shs.p","rb"))

In [ ]:
genres.head()

In [ ]:
covers=covers.merge(genres[['genre']],how='left',left_on='trackID',right_index=True)

In [ ]:
covers.head(15)

In [76]:
#pickle.dump(covers,open('data/covers_COMPLETE.p','wb'))
covers=pickle.load(open("data/covers_COMPLETE.p","rb"))

In [91]:
# The years recovered from the dataset are floats, so we need to strip their decimal part,
# and then we can take the last 4 characters on any date string to get the year.
covers["year"] = covers["date"].astype(str).replace("\.0","",regex=True).str[-4:]

In [92]:
covers

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id,country,genre,year
0,TRGDMZP128F42BC52B,ARB1DDF1187FB4FCFB,0,0,Louis Armstrong,Stardust,Louis Armstrong & His Orchestra,Unavailable,1988,0,Unknown,United States,Jazz,1988
1,TRCATYW12903D038FE,ARGJEEO1271F573FD6,0,0,Artie Shaw and his orchestra,Stardust,Artie Shaw and his orchestra,Unavailable,1988,0,Unknown,United States,Jazz,1988
2,TRVMZJZ128F4270CE4,ARY0HTV1187FB4A1B1,412972,0,Hoagy Carmichael,Star Dust,Hoagy Carmichael,English,1942,19677,Unknown,United States,Jazz,1942
3,TRKOINL128F42926C3,ARQ5FSZ1187B98AD74,0,0,Connee Boswell & Sy Oliver Orchestra,Star Dust,Connee Boswell & Sy Oliver Orchestra,Unavailable,NaN,0,Unknown,United States,NaN,nan
4,TROJZTF128F428B546,ARJN76O1187FB43C99,0,1,Ana Belén,Yo Vengo A Ofrecer Mi Corazon,Ana Belén,Unavailable,2001,0,Unknown,Spain,New Age,2001
5,TRYQEDQ128F427917C,ARS4KT21187B9B9438,0,1,Fito Paez,Yo Vengo A Ofrecer Mi Corazon,Fito Paez,Unavailable,NaN,0,Unknown,Argentina,Rock,nan
6,TRCKNGE128F92DA3F3,AR1CB5G1187B9AFB8E,16660,2,Electric Light Orchestra,Mr. Blue Sky,Electric Light Orchestra,English,1977,16660,16660,United Kingdom,Rock,1977
7,TRIOPLY128F423CFF3,ARKZJ301187FB521B2,551633,2,Lily Allen,Mr Blue Sky,Lily Allen,English,2009,16660,16660,United Kingdom,Pop,2009
8,TRWNDEU128F9329BF7,ARVZWQ31187B9B8946,354066,3,Liars,Mr Your On Fire Mr,Liars,English,"October 1, 2002",142889,354066,United States,Alternative,2002
9,TRYOPHS128F146DEFD,AR6NYHH1187B9BA128,354067,3,Yeah Yeah Yeahs,Mr. You're On Fire Mr.,Yeah Yeah Yeahs,English,"June 23, 2003",354066,354066,United States,Alternative,2003


In [96]:
covers[covers.original_id=='Unknown']

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id,country,genre,year
0,TRGDMZP128F42BC52B,ARB1DDF1187FB4FCFB,0,0,Louis Armstrong,Stardust,Louis Armstrong & His Orchestra,Unavailable,1988,0,Unknown,United States,Jazz,1988
1,TRCATYW12903D038FE,ARGJEEO1271F573FD6,0,0,Artie Shaw and his orchestra,Stardust,Artie Shaw and his orchestra,Unavailable,1988,0,Unknown,United States,Jazz,1988
2,TRVMZJZ128F4270CE4,ARY0HTV1187FB4A1B1,412972,0,Hoagy Carmichael,Star Dust,Hoagy Carmichael,English,1942,19677,Unknown,United States,Jazz,1942
3,TRKOINL128F42926C3,ARQ5FSZ1187B98AD74,0,0,Connee Boswell & Sy Oliver Orchestra,Star Dust,Connee Boswell & Sy Oliver Orchestra,Unavailable,NaN,0,Unknown,United States,NaN,nan
4,TROJZTF128F428B546,ARJN76O1187FB43C99,0,1,Ana Belén,Yo Vengo A Ofrecer Mi Corazon,Ana Belén,Unavailable,2001,0,Unknown,Spain,New Age,2001
5,TRYQEDQ128F427917C,ARS4KT21187B9B9438,0,1,Fito Paez,Yo Vengo A Ofrecer Mi Corazon,Fito Paez,Unavailable,NaN,0,Unknown,Argentina,Rock,nan
12,TRTUJKS128F4262F5F,ARJA1841187FB3A029,0,5,David Bowie,Moonage Daydream (Live),David Bowie,Unavailable,1996,0,Unknown,United Kingdom,Rock,1996
13,TRLLCAL128F428B903,ARP2RHS1187B991595,0,5,Zen Guerrilla,Moonage Daydream,Zen Guerilla,Unavailable,NaN,0,Unknown,Unavailable,Dance,nan
43,TRUATRC12903CAAF21,AR0RCMP1187FB3F427,0,16,Billie Jo Spears,Look What They've Done To My Song Ma,Billie Jo Spears,Unavailable,1996,0,Unknown,United States,Country,1996
44,TRPJVKE128F428AA27,ARCCN681187B9AFD73,0,16,Melanie,Look What They've Done To My Song_ Ma,Melanie,Unavailable,1999,0,Unknown,United States,Vocal,1999


In [103]:
# Find cliques where original_id='Unknown' and where there is at least one Nan year
nan_years=covers[covers.original_id=='Unknown'].groupby(['clique_id','year'],as_index=False)['clique_id'].agg({'freq':'count'})

In [111]:
nan_years[nan_years.year=='nan'].head()

,clique_id,year,freq
2,0,nan,1
4,1,nan,1
6,5,nan,1
11,21,nan,1
12,24,nan,2


In [112]:
#Number of cliques where nan present 
len(nan_years[nan_years.year=='nan'])

146

In [114]:
nan_years[nan_years.year=='nan'].clique_id.head()

2      0
4      1
6      5
11    21
12    24
Name: clique_id, dtype: int64

In [117]:
covers=covers[~covers.clique_id.isin(nan_years[nan_years.year=='nan'].clique_id)]

In [119]:
covers[covers.year=='nan'].shape

(187, 14)

In [213]:
original_song=pd.DataFrame(covers.original_id.unique())

In [214]:
original_song.shape

(3862, 1)

In [215]:
original_song['first_algo_result']='original'
original_song.columns=['original_id','first_algo_result']
original_song.set_index('original_id',inplace=True)

In [216]:
original_song.drop('Unknown',inplace=True)

In [217]:
original_song.shape

(3861, 1)

In [218]:
original_song.head()

,first_algo_result
original_id,
16660,original
354066,original
52010,original
9133,original
575,original


In [219]:
covers.shape

(17541, 15)

In [221]:
covers=covers.merge(original_song,how='left',left_on='shsPerf',right_index=True)

In [222]:
covers.head()

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id,country,genre,year,rank,first_algo_result
6,TRCKNGE128F92DA3F3,AR1CB5G1187B9AFB8E,16660,2,Electric Light Orchestra,Mr. Blue Sky,Electric Light Orchestra,English,1977,16660,16660,United Kingdom,Rock,1977,1,original
7,TRIOPLY128F423CFF3,ARKZJ301187FB521B2,551633,2,Lily Allen,Mr Blue Sky,Lily Allen,English,2009,16660,16660,United Kingdom,Pop,2009,2,NaN
8,TRWNDEU128F9329BF7,ARVZWQ31187B9B8946,354066,3,Liars,Mr Your On Fire Mr,Liars,English,"October 1, 2002",142889,354066,United States,Alternative,2002,1,original
9,TRYOPHS128F146DEFD,AR6NYHH1187B9BA128,354067,3,Yeah Yeah Yeahs,Mr. You're On Fire Mr.,Yeah Yeah Yeahs,English,"June 23, 2003",354066,354066,United States,Alternative,2003,2,NaN
10,TRQNZCE128E078A9C0,ARWILYB1187FB37DFE,52011,4,Bananarama,More_ More_ More,Bananarama,English,"March 20, 1993",52010,52010,United Kingdom,Pop,1993,2,NaN


In [225]:
year_nan=covers.groupby(['clique_id','year'],as_index=False)['clique_id'].agg({'freq':'count'})

In [231]:
year_nan[year_nan.year=='nan'].clique_id.head()

7      6
31    14
37    17
43    20
56    28
Name: clique_id, dtype: int64

In [232]:
second_algo=covers[~covers.clique_id.isin(year_nan[year_nan.year=='nan'].clique_id)]

In [236]:
second_algo.head()

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id,country,genre,year,rank,first_algo_result
6,TRCKNGE128F92DA3F3,AR1CB5G1187B9AFB8E,16660,2,Electric Light Orchestra,Mr. Blue Sky,Electric Light Orchestra,English,1977,16660,16660,United Kingdom,Rock,1977,1,original
7,TRIOPLY128F423CFF3,ARKZJ301187FB521B2,551633,2,Lily Allen,Mr Blue Sky,Lily Allen,English,2009,16660,16660,United Kingdom,Pop,2009,2,NaN
8,TRWNDEU128F9329BF7,ARVZWQ31187B9B8946,354066,3,Liars,Mr Your On Fire Mr,Liars,English,"October 1, 2002",142889,354066,United States,Alternative,2002,1,original
9,TRYOPHS128F146DEFD,AR6NYHH1187B9BA128,354067,3,Yeah Yeah Yeahs,Mr. You're On Fire Mr.,Yeah Yeah Yeahs,English,"June 23, 2003",354066,354066,United States,Alternative,2003,2,NaN
10,TRQNZCE128E078A9C0,ARWILYB1187FB37DFE,52011,4,Bananarama,More_ More_ More,Bananarama,English,"March 20, 1993",52010,52010,United Kingdom,Pop,1993,2,NaN


In [238]:
covers['second_algo_result']= np.where(((covers['rank']==1) & (covers.clique_id.isin(second_algo.clique_id))) , 'original', np.nan)

In [239]:
covers

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id,country,genre,year,rank,first_algo_result,second_algo_result
6,TRCKNGE128F92DA3F3,AR1CB5G1187B9AFB8E,16660,2,Electric Light Orchestra,Mr. Blue Sky,Electric Light Orchestra,English,1977,16660,16660,United Kingdom,Rock,1977,1,original,original
7,TRIOPLY128F423CFF3,ARKZJ301187FB521B2,551633,2,Lily Allen,Mr Blue Sky,Lily Allen,English,2009,16660,16660,United Kingdom,Pop,2009,2,NaN,nan
8,TRWNDEU128F9329BF7,ARVZWQ31187B9B8946,354066,3,Liars,Mr Your On Fire Mr,Liars,English,"October 1, 2002",142889,354066,United States,Alternative,2002,1,original,original
9,TRYOPHS128F146DEFD,AR6NYHH1187B9BA128,354067,3,Yeah Yeah Yeahs,Mr. You're On Fire Mr.,Yeah Yeah Yeahs,English,"June 23, 2003",354066,354066,United States,Alternative,2003,2,NaN,nan
10,TRQNZCE128E078A9C0,ARWILYB1187FB37DFE,52011,4,Bananarama,More_ More_ More,Bananarama,English,"March 20, 1993",52010,52010,United Kingdom,Pop,1993,2,NaN,nan
11,TRMBSQR128F92DF66E,ARPI2DX1187FB4CED4,52010,4,Andrea True Connection,More More More,Andrea True Connection,English,1976,52010,52010,United States,R&B/Soul,1976,1,original,original
14,TREVWUZ128F4263A9B,AR9UYPT1187B9AE833,45769,6,Hear'Say,Monday Monday,Hear'Say,English,2001,9133,9133,United Kingdom,NaN,2001,1,NaN,nan
15,TRGYREY128E0791913,ARQ294N1187FB53D2A,9133,6,The Mamas & The Papas,Monday_ Monday,The Mamas & The Papas,Unavailable,NaN,0,9133,United States,NaN,nan,2,original,nan
16,TRBLBSR128F425EBFE,AR36FFP1187B9926D7,59663,7,Silicon Teens,Memphis Tennessee,Silicon Teens,English,September 1980,575,575,United Kingdom,Electronic,1980,5,NaN,nan
17,TRLRWJK128F427D602,AR6NBDC1187FB4D96D,575,7,Chuck Berry,Memphis_ Tennessee,Chuck Berry,English,May 1959,575,575,United States,Rock,1959,1,original,original


In [246]:
pickle.dump(covers,open('data/covers_algo_results.p','wb'))
#covers=pickle.load(open("data/covers_COMPLETE.p","rb"))

In [241]:
second_algo_fail=covers.groupby(['clique_id','second_algo_result'],as_index=False)['clique_id'].agg({'freq':'count'})

In [242]:
second_algo_fail.head()

,clique_id,second_algo_result,freq
0,2,nan,1
1,2,original,1
2,3,nan,1
3,3,original,1
4,4,nan,1


In [244]:
len(second_algo_fail[(second_algo_fail.second_algo_result=='original') & (second_algo_fail.freq>1)])

380

In [145]:
covers[covers.original_id=='Unknown']

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id,country,genre,year,rank
43,TRUATRC12903CAAF21,AR0RCMP1187FB3F427,0,16,Billie Jo Spears,Look What They've Done To My Song Ma,Billie Jo Spears,Unavailable,1996,0,Unknown,United States,Country,1996,1
44,TRPJVKE128F428AA27,ARCCN681187B9AFD73,0,16,Melanie,Look What They've Done To My Song_ Ma,Melanie,Unavailable,1999,0,Unknown,United States,Vocal,1999,2
97,TRKFOUP128E0786014,ARUJZFJ1187B9B135F,0,40,Shania Twain,I'm Gonna Getcha Good!,Shania Twain / Alison Krauss and Union Station,Unavailable,2002,0,Unknown,Canada,Country,2002,1
98,TRRLKSI128F92D5D37,ARWQ4AM1187B9AD5F8,0,40,Jonas Brothers,I'm Gonna Getcha Good,Jonas Brothers,Unavailable,2009,0,Unknown,United States,NaN,2009,2
114,TRGEHDP128F148A1B4,AR9YWMS1187FB43A34,319149,48,Dinosaur Jr,Hot Burrito #2,Dinosaur Jr,English,2002,27841,Unknown,United States,Rock,2002,1
119,TRFIFNB128F92FB2A2,ARWEIPG1187FB57749,0,51,Claudine Longet,Hey That's No Way To Say Good-Bye,Claudine Longet,Unavailable,1972,0,Unknown,France,Rock,1972,3
120,TREQWRG128F92C49B4,ARTQ8QP1187FB3D220,68244,51,Leonard Cohen,Hey_ That's No Way To Say Goodbye,Leonard Cohen,English,"December 27, 1967",1855,Unknown,Canada,Alternative,1967,1
121,TRTCIBD12903C9A8E4,ARUXSE51187B997E35,390818,51,The Vogues,Hey_ That's No Way To Say Goodbye,The Vogues,English,May 1970,1855,Unknown,United States,NaN,1970,2
132,TRPEAEG128F1458AAB,ARYML1P1187B9AF39B,0,56,Devo,Working In A Coal Mine,Devo,Unavailable,1989,0,Unknown,United States,Alternative,1989,2
133,TRIHVRA128F92D2D01,ARKGX701187B993970,0,56,Lee Dorsey,Working In The Coalmine,Lee Dorsey,Unavailable,1988,0,Unknown,United States,Soul,1988,1


In [136]:
set_original(2)

(6    TRCKNGE128F92DA3F3
 Name: trackID, dtype: object, 'original')

In [127]:
covers_clique=covers.copy()
covers_clique.set_index(['clique_id','trackID'], inplace=True)
covers_clique

artistID  shsPerf  \
clique_id trackID                                           
2         TRCKNGE128F92DA3F3  AR1CB5G1187B9AFB8E    16660   
          TRIOPLY128F423CFF3  ARKZJ301187FB521B2   551633   
3         TRWNDEU128F9329BF7  ARVZWQ31187B9B8946   354066   
          TRYOPHS128F146DEFD  AR6NYHH1187B9BA128   354067   
4         TRQNZCE128E078A9C0  ARWILYB1187FB37DFE    52011   
          TRMBSQR128F92DF66E  ARPI2DX1187FB4CED4    52010   
6         TREVWUZ128F4263A9B  AR9UYPT1187B9AE833    45769   
          TRGYREY128E0791913  ARQ294N1187FB53D2A     9133   
7         TRBLBSR128F425EBFE  AR36FFP1187B9926D7    59663   
          TRLRWJK128F427D602  AR6NBDC1187FB4D96D      575   
          TRHFEEP12903CD3863  ARGYRE31187FB515A2    28082   
          TRWDJFG12903C94C9E  AR5ZI3K1187FB38BE0   150458   
          TRLQSWL128F932C80B  AR8JQ521187FB491AA    86557   
8         TRZWJEP128F92F3178  ARUL9Y61187FB3DF6B    68280   
          TRHFJNY128F428FC4B  AR5FP401187FB523C9    68275   
          TRMNZOQ128F426254A  AR9YR2T1187FB3E9EE    69922   
          TRYEKWE128F1459C42  AR6AK6B1187FB48EDD    68273   
          TRPTALT128F426F99F  ARQRDXP1187B98B9E6   305654   
9         TRQPBQV128F423C1E9  AREH7LP1187B9B7164    14943   
          TRTTEXX128F428BE1D  ARJVH1S1187FB378C3        0   
          TRYXXWR12903CA7191  ARMBTFC1187FB56343   160334   
10        TRVXQXV128F427863A  ARTO3L81187B9ACA8C    21513   
          TRZJWSX12903CA89C1  ARC1IHZ1187FB4E920    21514   
11        TREVQFK128F4267577  ARTQ8QP1187FB3D220    31718   
          TRFWVIP128EF3444F3  ARE66XI1187B9B8743    31719   
12        TRXZEBD128F4268BDB  AR33KSV1187FB4B900    88882   
          TRBZYQA128F9359288  AR20SBM1187B9966A7    88880   
13        TRTLSVW12903CFB4F5  ARWX5TK1187FB4CDBE    29876   
          TRJLBUB128F92E293E  ARN3SEQ1187FB47AA8   152956   
14        TRCNOGX128F92E9A8D  ARVODWZ1187B99E831        0   
...                                          ...      ...   
5842      TRDTJZI128EF34E78F  AR1BI4V1187B9937A9   184203   
          TRJUXRE128F92F88D4  ARHSO041187FB3D06D    33329   
          TRKFNUK128F428A62D  ARV3CRH1187B9A1B21    14338   
5843      TROWHKP12903CCC703  ARCGI111187B9B8B99    67211   
          TRELXLZ12903CB9DC5  ARYDLXP122988F049E    62368   
5844      TRPMGHH128E07917AC  ARF4L041187FB4D318    99651   
          TRMTFOJ12903CAFF57  AR3KVH41187FB3F71F    99652   
5845      TRUAXLK12903CD39B5  AREYJKQ1187B9AEAB5   104087   
          TRNNQSH128F4245865  AR4SL791187B9890BF        0   
          TRTWMBY128F9307A6D  ARSX6IX1187B9B640C    99655   
5846      TRITRRU128F423DC76  ARJ59K61187FB5B43E    99662   
          TRRCCUQ128F42449F7  ARNUU2R1187B9A316C    99675   
5847      TRCEMLE128F4281036  ARJ59K61187FB5B43E    99668   
          TRCGGYF128F9338B93  ARYDZKJ1187FB540C6    99680   
5848      TRAHFYD128F426BBAD  AR94ZOI1187FB46BDA   103980   
          TRRUFPY128F4261744  ARJWDFW1241B9C69AE   118487   
5849      TRTFWRF128F92E378E  ARX1CYE1187FB3A97B    99730   
          TRBXSHQ128F4278320  ARLM6I41187B9BA123    99731   
          TRGOMBU128F42895B9  AR14CJ91187FB3A994    99733   
5850      TRMCRMX128F4267AC3  ARSPUHE1187B99DB8A   125607   
          TRNBSWN128F1473B31  ARLG9UJ1187B9B6C67    24567   
5851      TRWAULS128F1474CBF  AR7GM0K1187B9B74B4    20304   
          TRZVRJL128F92E2984  ARDYR3C1187FB461CE     9982   
          TRMQPSQ128F426C548  ARAWPR61187B9B5A4C     9981   
5852      TRUZAVE128F426E391  AR3DLBB1187B98F3DF   106651   
          TRDNIRY128F425A5C8  ARDRJSP126E2B3BEF8   248918   
          TRSDUWY128F92DD39A  ARR176H1187FB4CDFE    99838   
          TRPVDKP128F934956E  ARELPXQ1187FB384FD   138704   
5853      TREMVLC128F92EFA95  ARBFDJW1187B9AD27A    29576   
          TRMTMWV128F92F6F78  ARDYR3C1187FB461CE     9986   

                                                              name  \
clique_id trackID                                                    
2         TRCKNGE128F92DA3F3              Electric Light

In [133]:
covers_clique.loc[2].original_id[0]

16660

In [134]:
covers[covers.shsPerf==covers_clique.loc[2].original_id[0]]

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id,country,genre,year,rank
6,TRCKNGE128F92DA3F3,AR1CB5G1187B9AFB8E,16660,2,Electric Light Orchestra,Mr. Blue Sky,Electric Light Orchestra,English,1977,16660,16660,United Kingdom,Rock,1977,1


In [123]:
# In each clique, we rank the versions by year to find the original
covers["rank"] = covers.groupby("clique_id")["year"].rank(method="dense",ascending=True).astype(int)

/Users/anaisladoy/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [124]:
covers

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id,country,genre,year,rank
6,TRCKNGE128F92DA3F3,AR1CB5G1187B9AFB8E,16660,2,Electric Light Orchestra,Mr. Blue Sky,Electric Light Orchestra,English,1977,16660,16660,United Kingdom,Rock,1977,1
7,TRIOPLY128F423CFF3,ARKZJ301187FB521B2,551633,2,Lily Allen,Mr Blue Sky,Lily Allen,English,2009,16660,16660,United Kingdom,Pop,2009,2
8,TRWNDEU128F9329BF7,ARVZWQ31187B9B8946,354066,3,Liars,Mr Your On Fire Mr,Liars,English,"October 1, 2002",142889,354066,United States,Alternative,2002,1
9,TRYOPHS128F146DEFD,AR6NYHH1187B9BA128,354067,3,Yeah Yeah Yeahs,Mr. You're On Fire Mr.,Yeah Yeah Yeahs,English,"June 23, 2003",354066,354066,United States,Alternative,2003,2
10,TRQNZCE128E078A9C0,ARWILYB1187FB37DFE,52011,4,Bananarama,More_ More_ More,Bananarama,English,"March 20, 1993",52010,52010,United Kingdom,Pop,1993,2
11,TRMBSQR128F92DF66E,ARPI2DX1187FB4CED4,52010,4,Andrea True Connection,More More More,Andrea True Connection,English,1976,52010,52010,United States,R&B/Soul,1976,1
14,TREVWUZ128F4263A9B,AR9UYPT1187B9AE833,45769,6,Hear'Say,Monday Monday,Hear'Say,English,2001,9133,9133,United Kingdom,NaN,2001,1
15,TRGYREY128E0791913,ARQ294N1187FB53D2A,9133,6,The Mamas & The Papas,Monday_ Monday,The Mamas & The Papas,Unavailable,NaN,0,9133,United States,NaN,nan,2
16,TRBLBSR128F425EBFE,AR36FFP1187B9926D7,59663,7,Silicon Teens,Memphis Tennessee,Silicon Teens,English,September 1980,575,575,United Kingdom,Electronic,1980,5
17,TRLRWJK128F427D602,AR6NBDC1187FB4D96D,575,7,Chuck Berry,Memphis_ Tennessee,Chuck Berry,English,May 1959,575,575,United States,Rock,1959,1


In [ ]:
unknown_originals = covers[covers["rank"] == 1][covers[covers["rank"] == 1].duplicated(subset="clique_id",keep=False)]

In [ ]:
merge_covers.head(10)

In [1982]:
#Save cover dataFrame in pickle file
pickle.dump(original_song_df,open('data/original_song_df.p','wb'))
pickle.dump(covers,open('data/covers_merged_new.p','wb'))

In [1928]:
original_song_df[(original_song_df.duplicated(subset='original_id')==True) & (original_song_df.original_id!='Unknown')].head()

,clique_id,original_id
600,600,102705
889,889,19517
1173,1173,1108
1510,1510,11762
1544,1544,12078


In [1894]:
covers[covers.clique_id==5158]

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id
16107,TRPMNOC128F426D66A,ARGETJ81187B9B4F2F,7638,5158,Son House,Death Letter,Son House,English,1965,7637,7637
16108,TRKNYEY128F4266D71,ARGRDHI1187B9B93F0,32096,5158,James Blood Ulmer,Death Letter,James Blood Ulmer,English,"September 4, 2001",7637,7637
16109,TRATROO128F42544AB,ARZHZ8X1187B9A5C1B,32101,5158,Ace Moreland,Death Letter Blues,Ace Moreland,English,"November 7, 1995",7637,7637
16110,TRAUGBG128F932B2A2,AR6V68G1187B98DCC1,32102,5158,Geoff And Maria Muldaur,Death Letter Blues (Album Version),Geoff & Maria Muldaur,English,1969,7637,7637
16111,TRDDNWC128F9311E1C,ARGETJ81187B9B4F2F,7637,5158,Son House,My Black Mama,Son House,English,February 1931,145153,7637


In [1904]:
covers[covers.clique_id==1228]

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id
2981,TRCOPSN12903C96988,AR6EAZK1187B98E869,114688,1228,Dee Dee Bridgewater,Cherokee,Dee Dee Bridgewater,English,1987,114679,Unknown
2982,TRVTYGT128F42B9600,ARCFDW71187B9BA40B,114765,1228,Tal Farlow,Cherokee,Tal Farlow,Missing,1955,114679,Unknown
2983,TRRJQGY128F9339EC6,ARCLE5B1187B98D185,114766,1228,Danny Gatton,Cherokee,Danny Gatton,Missing,1987,114679,Unknown
2984,TRJJKML128F4231B3A,ARSYWV7119B3403694,122352,1228,Jorgen Ingmann,Cherokee (LP Version),Jorgen Ingmann,Missing,1961,114679,Unknown


In [1929]:
covers[covers.clique_id==3584]

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id
11022,TRGPQJA128F93291EA,ARRU6BI1187FB5129D,405,3584,Jimmy Ruffin,What Becomes of the Brokenhearted,Jimmy Ruffin,English,"June 3, 1966",56770,405
11023,TRYENMJ128F9321F76,ARUAKBD1187B9A2114,76915,3584,Jimmy Barnes,What Becomes Of The Broken Hearted,Jimmy Barnes,English,2000,405,405
11024,TRKMNEL128F425A058,ARKOSUV1187FB3C1FC,59901,3584,Andy Abraham,What Becomes Of The Brokenhearted,Andy Abraham,English,"November 13, 2006",405,405
11025,TRZJUHJ128F429A6BC,AREIZH41187B9A108D,48402,3584,Joan Osborne,What Becomes of the Brokenhearted (Bonus Track),Joan Osborne,English,September 2002,405,405
11026,TRVCJBN128F9321008,AR8L6W21187B9AD317,11406,3584,Diana Ross,What Becomes Of The Brokenhearted,Diana Ross & The Supremes,English,"May 26, 1969",405,405


In [1931]:
covers[covers.clique_id==1754]

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id
4722,TRIUGQT128F1463A47,ARHA1XR1187B9B5940,25998,1754,The Dickies,Sounds Of Silence,The Dickies,English,December 1978,623,623
4723,TRMEREM128EF34A4C1,AR0XFXQ1187B98C49F,624,1754,The Bachelors,The Sound Of Silence,The Bachelors,English,March 1966,623,623
4724,TRKEHUH12903CAC508,AR5NU351187B9AE701,120671,1754,Sharleen Spiteri,The Sound Of Silence,Sharleen Spiteri,English,"March 1, 2010",623,623
4725,TRHPXVG128F4237EB1,ARUGX5X1187FB468C4,148152,1754,Dino,La Tua Immagine,Dino,Italian,1968,623,623
4726,TRTJPUZ128F4269A44,ARIWB161187B9AA1D5,623,1754,Simon & Garfunkel,The Sounds Of Silence,Simon & Garfunkel,English,"October 19, 1964",623,623


In [1930]:
covers[covers.original_shsPerf==623]

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id
637,TRQVAJJ128F4284ADC,ARIWB161187B9AA1D5,368538,289,Simon & Garfunkel,The Sound Of Silence,Simon & Garfunkel,English,August 1965,623,623
638,TRVCWFM12903CFEA12,AR8S6R11187B9B0407,25975,289,Nevermore,Sound Of Silence,Nevermore,English,"October 17, 2000",623,623
4722,TRIUGQT128F1463A47,ARHA1XR1187B9B5940,25998,1754,The Dickies,Sounds Of Silence,The Dickies,English,December 1978,623,623
4723,TRMEREM128EF34A4C1,AR0XFXQ1187B98C49F,624,1754,The Bachelors,The Sound Of Silence,The Bachelors,English,March 1966,623,623
4724,TRKEHUH12903CAC508,AR5NU351187B9AE701,120671,1754,Sharleen Spiteri,The Sound Of Silence,Sharleen Spiteri,English,"March 1, 2010",623,623
4725,TRHPXVG128F4237EB1,ARUGX5X1187FB468C4,148152,1754,Dino,La Tua Immagine,Dino,Italian,1968,623,623
4726,TRTJPUZ128F4269A44,ARIWB161187B9AA1D5,623,1754,Simon & Garfunkel,The Sounds Of Silence,Simon & Garfunkel,English,"October 19, 1964",623,623


In [1899]:
covers

,trackID,artistID,shsPerf,clique_id,name,title,artist,language,date,original_shsPerf,original_id
0,TRGDMZP128F42BC52B,ARB1DDF1187FB4FCFB,0,0,Louis Armstrong,Stardust,Louis Armstrong & His Orchestra,Unavailable,1988,0,Unknown
1,TRCATYW12903D038FE,ARGJEEO1271F573FD6,0,0,Artie Shaw and his orchestra,Stardust,Artie Shaw and his orchestra,Unavailable,1988,0,Unknown
2,TRVMZJZ128F4270CE4,ARY0HTV1187FB4A1B1,412972,0,Hoagy Carmichael,Star Dust,Hoagy Carmichael,English,1942,19677,Unknown
3,TRKOINL128F42926C3,ARQ5FSZ1187B98AD74,0,0,Connee Boswell & Sy Oliver Orchestra,Star Dust,Connee Boswell & Sy Oliver Orchestra,Unavailable,NaN,0,Unknown
4,TROJZTF128F428B546,ARJN76O1187FB43C99,0,1,Ana Belén,Yo Vengo A Ofrecer Mi Corazon,Ana Belén,Unavailable,2001,0,Unknown
5,TRYQEDQ128F427917C,ARS4KT21187B9B9438,0,1,Fito Paez,Yo Vengo A Ofrecer Mi Corazon,Fito Paez,Unavailable,NaN,0,Unknown
6,TRCKNGE128F92DA3F3,AR1CB5G1187B9AFB8E,16660,2,Electric Light Orchestra,Mr. Blue Sky,Electric Light Orchestra,English,1977,16660,16660
7,TRIOPLY128F423CFF3,ARKZJ301187FB521B2,551633,2,Lily Allen,Mr Blue Sky,Lily Allen,English,2009,16660,16660
8,TRWNDEU128F9329BF7,ARVZWQ31187B9B8946,354066,3,Liars,Mr Your On Fire Mr,Liars,English,"October 1, 2002",142889,354066
9,TRYOPHS128F146DEFD,AR6NYHH1187B9BA128,354067,3,Yeah Yeah Yeahs,Mr. You're On Fire Mr.,Yeah Yeah Yeahs,English,"June 23, 2003",354066,354066


In [ ]:
original_song.shape

In [1898]:
cover_new=covers.copy()

In [ ]:
cover_new[]

In [ ]:
covers['original_shsPerf'] = np.where(((covers['original_shsPerf']=='Unavailable') | (covers['original_shsPerf']=='Missing')), 0, covers['original_shsPerf'])
kidheart break

We have only 345 tracks with no informations about the year.

2/ Some original performance that were found in the SHS website don't appear in the clique so need to add these tracks to our dataframe.
- See for each clique if there is at least one original song defined
- See if the original song is unique (in each clique)
- Check if we have information about the orginal song in our dataframe or if we need to web-scrap from SHS website
- REMOVE DUPLICATES IN TRACKS (same shs perf)

In [ ]:
merge_covers

In [ ]:
covers

In [ ]:
#At least one original song in each clique ?
#Replace missing values by Nan in the original_shsPerf column (to don't count them as unique values)
covers.original_shsPerf.replace(['Missing','Unavailable'],np.NaN,inplace=True)
count_unique=covers.groupby('clique_id').original_shsPerf.nunique(dropna=True)

In [ ]:
#Is the original song unique in each clique ?
count_unique[count_unique>1].head()

In [ ]:
count_unique[count_unique==0].index

In [ ]:
count_tracks_clique=covers.groupby('clique_id').size()

In [ ]:
count_tracks_clique[count_tracks_clique==1]

In [ ]:
#All the cliques having more than 1 Original Performance
covers[covers.clique_id.isin(count_unique[count_unique>1].index)==True]

In [ ]:
#All the cliques having only NaN values for Original Performance
covers[covers.clique_id.isin(count_unique[count_unique==0].index)==True]

In [ ]:
#Take the clique id we defined as id of the dataframe (not unique index for now)
covers.set_index('clique_id',inplace=True)
covers.sort_index(inplace=True)

In [ ]:
covers.head()

In [ ]:
covers.loc[covers[(covers.year.isnull()) & ((covers.date!='Missing') & (covers.date!='Unavailable'))].loc,

In [ ]:
covers[(covers.year=='Missing') | (covers.year=='Unavailable')].shape

In [ ]:
#Number of cliques with no original song found
len(count_unique[count_unique==0])

In [ ]:
#
shsPerfUnique=pd.DataFrame(covers.shsPerf.unique())
originalPerfUnique=pd.DataFrame(covers.originalPerfUnique.unique())
merge=originalPerfUnique.merge(shsPerfUnique,how='left')
merge

In [ ]:
covers

Here is the information we obtained for now. We noticed several problems that needs to be fixed later :
- The track year (year) is sometimes different form the released date (date) we've extracted from the SHS website, we will prefer the data information found in the SHS website.
- Some languages and some dates are missing (we will consider droping these covers or restrict our analyse concerning these parameters to only a subset of cliques).
- Some original performance that were found in the SHS website don't appear in the clique so need to add these tracks to our dataframe (find them in the cluster or webscrap directly to the SHS website)
- We may still have a problem to find the ranking (just discriminate the original track and do not sort the music covers).

The work for this part will be to extend the analysis to the entire cover dataframe, resolve problems cited above and finish the multilevel indexing using ranking according the date in each clique. The API request is limited for the Second Hand Songs (SHS) website to 1000 requests per hour. Due to the large number of requests needed (668 to resolve the missing SHS problem and 18196 to find the Language/Year/Original Song), we'll maybe ask to the SHS team an exception to remove this limitation.

Then, the goal is to add to this dataframe the informations we've extracted through the cluster (tempo, song hotness), through the LastFM dataset (genre) and through the artist_location table (country of the artist) with the methods we describe in the next section. 

Thus, we'll have all the informations required to start the analysis of our music covers.

In [ ]:


#Compute the order of songs for each clique
#covers['rank']=covers.groupby('clique_id')['year'].rank(method='dense',ascending=True).astype('int')

### 4. Access to files (tempo / dancability)

We open the first file of the subset, to check what the HDF5 keys are and then we read each of them.

In [ ]:
with pd.HDFStore("data/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5") as hdf:
    print(hdf.keys())

In [ ]:
pd.read_hdf("data/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5","/analysis/songs")

In [ ]:
pd.read_hdf("data/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5","/metadata/songs")

In [ ]:
pd.read_hdf("data/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5","/musicbrainz/songs")

We only need to extract <tt>tempo</tt> and <tt>song_hotttnesss</tt>, here is an example of how to do that on the subset :

In [ ]:
tempo = []
hotness = []

files = glob.glob("data/MillionSongSubset/data/A" + "/[A-Z]/[A-Z]/*")
for f in files:
    tempo.append(pd.read_hdf(f,"/analysis/songs")["tempo"][0])
    hotness.append(pd.read_hdf(f,"/metadata/songs")["song_hotttnesss"][0])

In [ ]:
tempo = np.asarray(tempo)
hotness = np.asarray(hotness)
print(tempo)
print(hotness)

In [ ]:
print("Number of tracks =", len(files))
print("with missing hotness values =", np.sum(np.isnan(hotness)))

We already got 3268 unknown hotness values and we only tested on a subset of 7620 song, so we can expect to have that information for only a little over half of our final dataset. Maybe we won't use it.

Once all the files are accessible on the cluster, we will have to go through our SHS dataset and get those attributes for each track_id.
We will do so in the following way : (the paths are just examples on the subset)

In [ ]:
tempo = []
hotness = []

my_file = Path("/path/to/file")
for track in covers["trackID"]:
    folder1 = track[2]
    folder2 = track[3]
    folder3 = track[4]
    folder_path = "data/MillionSongSubset/data/" + folder1 + "/" + folder2 + "/" + folder3 + "/"
    track_path = folder_path + track + ".h5"
    if Path(track_path).exists(): #to delete later
        tempo.append(pd.read_hdf(track_path,"/analysis/songs")["tempo"][0])
        hotness.append(pd.read_hdf(track_path,"/metadata/songs")["song_hotttnesss"][0])

In [ ]:
print(len(tempo))
print(np.sum(~np.isnan(hotness)))

Unsurprisingly, we only found 204 of those tracks in the subset and 128 of them have a hotness value.

### 5. Determine artist location for spatial analysis

In [ ]:
#Load Additional files
#unique_artists=pd.read_csv('data/AdditionalFiles/unique_artists.txt',delimiter='<SEP>',engine='python',header=None,index_col=0,names=['artistID','artistMID','randomTrack','name'])
unique_artists=pd.read_csv('data/AdditionalFiles/unique_artists.txt',delimiter='<SEP>',engine='python',header=None,index_col=0,names=['artistID','artistMID','randomTrack','name'])
artist_location=pd.read_csv('data/AdditionalFiles/artist_location.txt',delimiter='<SEP>',engine='python',header=None,index_col=0,names=['artistID','lat','long','name','location'])
artist_location.head()

We now load a subset of Second Hand Song

In [ ]:
def read_shs_files(pathToFile):
    f = open(pathToFile)
    s = StringIO()
    cur_ID = None
    for ln in f:
        if not ln.strip():
                continue
        if ln.startswith('%'):
                cur_ID = ln.replace('\n','<SEP>',1)
                continue
        if cur_ID is None:
                print ('NO ID found')
                sys.exit(1)
        s.write(cur_ID + ln)
    s.seek(0)
    df = pd.read_csv(s,delimiter='<SEP>',engine='python',header=None,names=['shsID','trackID','artistID','shsPerf'])
    return df[['trackID', 'artistID', 'shsPerf']]

We retrieve the artists' names using the unique_artists.txt file and we assign a location for each track using the artist_location.txt.

In [ ]:
def get_location(x) : 
    if x in artist_location.index:
        return artist_location.get_value(x, 'location')
    else : 
        return np.nan
    
data=read_shs_files('data/SHS_testset.txt')
data['artist'] = data['artistID'].map(lambda x : unique_artists.get_value(x, 'name'))
data['location'] = data['artistID'].map(lambda x : get_location(x))
data.head()

We now create the function finding the country for each location. In order to do that we wille use three different python packages : pycountry, us, and geopy, as geopy.geocoders does not support too much requests. 

- First, we will use the pycountry package to extract countries if location contains one. 


- If we didn't match any country in pycountry, we will use the us package to check if a us state is present in the location. From the data, we have observed that if the location refer to a us state, the location is either only defined by the state, or the state is the last element of the location.


- If the two precedent methods does not succeed, we will use the geopy.geocoders package, using Nominatim( ).


- We will manually define countries for some location as they are sometimes mispelled, troncated or refer to a website link.

In [ ]:
geolocator = Nominatim()

def get_country(x):
    if x == np.nan:
        return x
    x = x.replace("-", ",")
    for c in pycountry.countries:
        if "England" in x or "UK" in x: 
            return "United Kingdom"
        elif c.name.lower() in x.lower():
            return c.name
    refactorlast = x.split(",")[-1].replace(" ", "")
    refactorfirst = x.split(",")[0]
    usstatelast = us.states.lookup(refactorlast)
    usstatefirst = us.states.lookup(refactorfirst)
    if usstatelast != None or usstatefirst != None:
        return "United State of America"
    elif x == "Swingtown":
        return "United State of America"
    elif x == "<a href=\"http://billyidol.net\" onmousedown='UntrustedLink.bootstrap($(this), \"fc44f8f60d13ab68c56b3c6709c6d670\", event)' target=\"_blank\" rel=\"nofollow\">http://billyidol.net</a>":
        return "United Kingdom"
    elif x == "Lennox Castle, Glasgow" or x == "Knowle West, Bristol, Avon, Engla"\
        or x == "Goldsmith's College, Lewisham, Lo" or x == "Julian Lennon&#039;s Official Facebook Music Page"\
        or x == "Sydney, Moscow, Pressburg" or x == "Penarth, Wales to Los Angeles" or x == "Leicester, Leicestershire, Englan":
        return "United Kingdom"
    elif x == "Vancouver, British Columbia, Cana":
        return "Canada"
    elif x == "Washington DC" or x == "Philladelphia" or "New Jersey" in x:
        return "United State of America"
    elif "Czechoslovakia" in x :
        return "Česko"
    elif x == "Jaded Heart Town":
        return "Germany"
    elif x == "RU" or x == "Russia":
        return "Russia"
    else :
        location = geolocator.geocode(x, timeout=None)
        return location.address.split(",")[-1]

In [ ]:
#data['country'] = data['location'].map(lambda x : get_country(x))

The only problem with geopy is that it returns a country in its native language. To uniform our data, we create a function that translates manually the countries in English.

In [ ]:
def rename(x):
    if "België - Belgique - Belgien" in x:
        return "Belgium"
    elif "Brasil" in x:
        return "Brazil"
    elif "United State" in x:
        return "United States of America"
    elif "Italia" in x:
        return "Italy"
    elif "Norge" in x:
        return "Norway"
    elif "España" in x:
        return "Spain"
    elif "Nederland" in x :
        return "Netherlands"
    elif "Suomi" in x :
        return "Finland"
    elif "Sverige" in x :
        return "Sweden"
    elif "UK" in x :
        return "United Kingdom"
    elif x[0] == " ":
        return x[1:]
    else : 
        return x

In [ ]:
#data['country'] = data['country'].map(lambda x : rename(x))
#pickle.dump(data, open( "data.p", "wb" ) )
data_country = pickle.load(open("data/data_country.p", "rb"))

In [ ]:
data_country.head()

### 6. Addition of the genre for each track (Use of LastFM dataset and external website for genre listing)

To find the genre of a song, we will use the LastFM dataset that contains a list a tags for each song.
Since the dataset is from the MillionSongDataset, we will not use all of the available tracks from LastFM but, but only the ones contained in the SecondHandSong dataset.

In [ ]:
# Loading the files if they are in the SecondHandSong dataset and create the dataframe
covers_df = pickle.load(open("data/covers.p","rb"))
list_tracks = covers_df.trackID
test_path = "../../lastfm_test"
train_path = "../../lastfm_train"

genre_df = pd.DataFrame()
def create_dataFrame(genre_df):
    for track in list_tracks:
        folder1 = track[2]
        folder2 = track[3]
        folder3 = track[4]
        folder_path = "/" + folder1 + "/" + folder2 + "/" + folder3 + "/"
        track_path = folder_path + track + ".json"
        if glob.glob(train_path + track_path) != []:
                genre_df = genre_df.append(pd.DataFrame.from_dict(json.load(open(train_path + track_path)), orient="index").transpose())
        elif glob.glob(test_path + folder_path + track) != []:
                genre_df = genre_df.append(pd.DataFrame.from_dict(json.load(open(test_path + track_path)), orient="index").transpose())
    genre_df = genre_df.reset_index()
    return genre_df

#tracks_with_tags = create_dataFrame(genre_df)
tracks_with_tags = pickle.load(open("tracks_with_tags", "rb"))

We now list the unique tags in the resulting dataframe. Due to a time limit for the computation of the matching, we will first test on a subset.

In [ ]:
tags = list()
for i in range (0,1000):
    tags = tags + tracks_with_tags.tags[i]
    
tags = np.unique(tags).tolist()

A lot of tags contains useless information, thus we first proceed to a pre-cleaning.

In [ ]:
clean_tags = {}
def clean_tag(x):
    clean = x.replace("ooo", "")
    clean = clean.replace("-o", "")
    clean = clean.replace("o-", "")
    clean = clean.replace("- ", "")
    clean = clean.replace("-", "")
    clean_tags[x] = clean
for t in tags:
    clean_tag(t)

In order assign a genre to each song, we will use their different tags and try to match it with a list of genre obtained by webscrapping the http://www.musicgenreslist.com website. For more details on the webscrapping see the notebook Genre Webscrapping.ipynb.

In [ ]:
map_genres = pickle.load(open("data/map_genres", "rb"))
all_genres = pickle.load(open("data/all_genres.p", "rb"))

We then use the Sequence Matcher package to match tags to the web-scrapped genres.

In [ ]:
threshold = 0.80
def match_genres():
    i = 0
    genre_map = {}
    no_match = list()
    for ind in range(0,len(tags)):
        name1 = tags[ind]
        if i%1000 == 0:
            print(i)
        if clean_tags[name1] == "":
            genre_map[name1] = np.nan
        best_ratio = 0
        match = ""
        for name2 in map_genres.keys():
            if name2.lower() in name1.lower():
                for subgenre in map_genres[name2]:
                    ratio = SequenceMatcher(None,name1.lower(),name2.lower()).ratio()
                    if ratio > best_ratio:       # we find the maximum similarity
                        best_ratio = ratio
                        match = name2
                if (best_ratio > threshold):     # if it's superior to our threshold we add that couple to the mapping
                    genre_map[name1] = match
                else:
                    genre_map[name1] = name2
        if match == "":
            for subgenre in all_genres:
                ratio = SequenceMatcher(None,name1.lower(),name2.lower()).ratio()
                if ratio > best_ratio:       # we find the maximum similarity
                    best_ratio = ratio
                    match = name2
            if (best_ratio > threshold):     # if it's superior to our threshold we add that couple to the mapping
                genre_map[name1] = match
            else :
                genre_map[name1] = np.nan
        i = i+1
    return (genre_map, no_match)

### 4. Access to files (tempo / dancability)